## 빅데이터 자습

### 다나와에서 식단관리 도시락 가격 대비 단백질과 열량 정보 표기

#### 데이터 수집

##### 셀레니움 사용

In [1]:
## 필수 라이브러리 사용등록
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib import parse
from tqdm import tqdm

In [13]:
## 웹드라이버로 크롬오픈
import time


driver = webdriver.Chrome()
url = r'https://prod.danawa.com/list/?cate=16245645&15main_16_02'
driver.get(url)
time.sleep(5.0)

In [14]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [15]:
soup

<html class="js no-touch" lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml"><head>
<title>식단관리 도시락 : 다나와 가격비교</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="다나와 가격비교 식단관리 도시락 상품리스트" name="apple-mobile-web-app-title"/>
<meta content="식단관리 도시락 : 다나와 가격비교" property="og:title"/>
<meta content="식품/유아/완구&gt;헬스/다이어트식품&gt;식단관리 도시락 가격비교 리스트 입니다. 다나와의 가격비교 서비스로 한눈에 비교하고 저렴하게 구매하세요." property="og:description"/>
<meta content="//img.danawa.com/new/noData/img/noImg_360.gif" property="og:image"/>
<meta content="//img.danawa.com/new/noData/img/noImg_360.gif" property="twitter:image"/>
<meta content="식단관리 도시락 : 다나와 가격비교" name="Title"/>
<meta content="식품,가공식품,즉석밥,즉석국" name="Keywords"/>
<meta content="식품/유아/완구&gt;헬스/다이어트식품&gt;식단관리 도시락 가격비교 리스트 입니다. 다나와의 가격비교 서비스로 한눈에 비교하고 저렴하게 구매하세요." name="Description"/>
<meta content="cfaabed510e5fa8a119d7488039f32e9a4ca65ca" name="naver-site-verification"/>
<m

In [16]:
prodItems = soup.select('ul.product_list > li.prod_item')

In [17]:
len(prodItems)

31

In [29]:
prodItems[0].select('p.prod_name > a')[0].text.strip()

'시즌2 오늘한끼 도시락 세트'

In [60]:
prodItems[0].select('div.spec_list')[0].text.strip()

'식단관리 도시락 / 밥 종류: 볶음밥 / 형태: 분리식 / 구성: 6종류 / 포장형태: 용기형 / 냉동보관 / 칼로리조절 / [영양정보] 표시기준량: 250g / 열량: 373~490kcal / 단백질: 23~35%'

In [110]:
prodItems[0].select('p.price_sect > a > strong')

[<strong>26,890</strong>, <strong>32,900</strong>]

In [ ]:
for won in prodItems:
    

##### 다나와 도시락 웹크롤링 다시
- 가격 외에는 안 들어오도록 변환

In [78]:
## 검색어, 페이지를 변경하면서 URL 생성함수
def getSearchPageUrl(keyword, page):
    ecKeyword = parse.quote(keyword)
    url = f'https://search.danawa.com/dsearch.php?query={ecKeyword}&originalQuery={ecKeyword}&previousKeyword={ecKeyword}&checkedInfo=N&volumeType=allvs&' + \
            f'page={page}&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&' + \
            'recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&' + \
            'quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'
    return url

In [79]:
## 상품정보 추출하는 함수
def getProdItems(prodItem):
    prodData = []

    for prodItem in prodItems:
        try:
            prodName = prodItem.select('p.prod_name > a')[0].text.strip() # 상품명 가져오기
            specList = prodItem.select('div.spec_list')[0].text.strip().replace('\t', '') # 상품 정보 가져오기
            if int(prodItems.select('p.price_sect>a>strong')[0].text.strip().replace(',', ''))
                price = prodItem.select('input')[1].get('value') # 최저가 가져오기
            else:
                price = 0 # 문자열 들어온 것 막음
            
            prodData.append([prodName, specList, price])
            
        except:
            pass
    return prodData